In [37]:
import databento as db
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [38]:
client = db.Historical("db-QNue9tcjaXcJGDXRTMnSPAvrF8V7c")



In [39]:
datasets = client.metadata.list_datasets()
print(datasets)

['ARCX.PILLAR', 'DBEQ.BASIC', 'EPRL.DOM', 'EQUS.SUMMARY', 'GLBX.MDP3', 'IEXG.TOPS', 'IFEU.IMPACT', 'NDEX.IMPACT', 'OPRA.PILLAR', 'XASE.PILLAR', 'XBOS.ITCH', 'XCHI.PILLAR', 'XCIS.TRADESBBO', 'XNAS.BASIC', 'XNAS.ITCH', 'XNYS.PILLAR', 'XPSX.ITCH']


In [40]:
#start_date a week from today
start_date = datetime.now() - timedelta(days=7)
#end date yesterday
end_date = datetime.now() - timedelta(days=1)
symbols = ["AAPL"]

df = client.timeseries.get_range(
    dataset="XNAS.ITCH",
    schema="mbp-10",
    symbols=symbols,
    start=start_date,
    end=end_date,
    limit=10_000,
).to_df()

df.head()

,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,...,ask_sz_08,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09,symbol
ts_recv,,,,,,,,,,,,,,,,,,,,,
2025-01-08 23:54:08.077796070+00:00,2025-01-08 23:54:08.077629708+00:00,10,2,38,A,A,0,242.35,2,130,...,1325,1,12,241.56,243.04,3,3,1,1,AAPL
2025-01-08 23:55:14.689530891+00:00,2025-01-08 23:55:14.689364413+00:00,10,2,38,C,A,6,242.85,4,130,...,3,1,1,241.56,243.10,3,4,1,1,AAPL
2025-01-08 23:55:32.518177375+00:00,2025-01-08 23:55:32.518010407+00:00,10,2,38,T,B,0,242.35,2,130,...,3,1,1,241.56,243.10,3,4,1,1,AAPL
2025-01-08 23:55:32.518177375+00:00,2025-01-08 23:55:32.518010407+00:00,10,2,38,C,A,0,242.35,2,130,...,4,1,1,241.56,243.15,3,29,1,1,AAPL
2025-01-08 23:56:59.849919431+00:00,2025-01-08 23:56:59.849746166+00:00,10,2,38,T,N,0,242.31,10,0,...,4,1,1,241.56,243.15,3,29,1,1,AAPL


In [41]:
def create_snapshot(df, ti):
  depth = 10
  ask_prices = []
  bid_prices = []

  ask_vols = []
  bid_vols = []

  for i in range(depth):
    bv = f'bid_sz_0{i}'
    av = f'ask_sz_0{i}'
    bp = f'bid_px_0{i}'
    ap = f'ask_px_0{i}'
    
    ask_vols.append(df.iloc[ti][av])
    bid_vols.append(df.iloc[ti][bv])
    bid_prices.append(df.iloc[ti][bp])
    ask_prices.append(df.iloc[ti][ap])

  #bid_vols = np.array(bid_vols) * -1
  return ask_vols, bid_vols, ask_prices, bid_prices

In [42]:
def create_orderbook():
  times = []
  ask_vols_t = []
  bid_vols_t = []
  ask_prices_t = []
  bid_prices_t = []
  for ti in range(0, len(df), 10):
    ask_vols, bid_vols, ask_prices, bid_prices = create_snapshot(df, ti)
    ask_vols_t.append(ask_vols)
    bid_vols_t.append(bid_vols)
    ask_prices_t.append(ask_prices)
    bid_prices_t.append(bid_prices)
    snapshot_time = df.iloc[ti]['ts_in_delta']
    times.append([snapshot_time] * 10)

  apx = np.array(ask_prices_t)
  bpx = np.array(bid_prices_t)
  avx = np.array(ask_vols_t)
  bvx = np.array(bid_vols_t)
  times = np.array(times)

  avc =  np.cumsum( bvx, axis=1 )
  bvc = np.fliplr( np.cumsum( np.fliplr(bvx), axis=1 ) )

  return apx, bpx, avc, bvc, times


In [43]:
apx.shape

(1000, 10)

In [44]:
apx, bpx, avc, bvc, times = create_orderbook()
op = 0.8
inds = np.arange(0, 1000, 10)

fig = go.Figure(data = [
  go.Surface(x=apx, 
             y=np.arange(len(apx)), 
             z=avc,
             colorscale = 'Viridis', 
             opacity=op
             )])

fig.add_surface(
    x=bpx, 
    y=np.arange(len(bpx)), 
    z=bvc,
    colorscale = 'agsunset', 
    opacity=op
  )

fig.update_layout(
    title="Orderflow Ravine for $AAPL",
    scene=dict(
        xaxis_title="Price",
        yaxis_title="Time",
        zaxis_title="CumulativeVolume"
    )
)

fig.show()